# Geostrophic velocity calculations on an unstructured mesh
This notebook is used to compute geostrophic velocities from sea surface height data generated by FESOM. Adapted from original code by Qiang Wang.

In [ ]:
import numpy as np
import xarray as xra
import math
from joblib import Parallel, delayed

In [ ]:
#get mesh information
fid=open('nod2d.out')
nodes=np.genfromtxt(fid,delimiter=None,skip_header=1)
n2d=nodes[:,0]
fid.close()
xcoord=nodes[:,1]
ycoord=nodes[:,2]
icoord=nodes[:,3]
xr=nodes[:,1]
yr=nodes[:,2]

fid=open('elem2d.out')
elem=np.genfromtxt(fid,delimiter=None,skip_header=1)
el2d=elem[:,0]
elem_back=elem.astype(int)
elem=elem.astype('int32')
fid.close()

# Derivative for triangles
Calculate the zonal and meridional gradients of sea surface height on triangular elements

In [ ]:
xcoord=xr
ycoord=yr

n2d=len(xcoord)
el2d=elem_back.shape[0]

r_earth=6367500
domain_length=math.radians(360)

bafux_2d=np.zeros(shape=(3,el2d))
bafuy_2d=np.zeros(shape=(3,el2d))
voltriangle=np.zeros(shape=(1,el2d))
full_cluster_area=np.zeros(shape=(1,n2d))

derivative_stdbafu_x_2D= np.zeros(shape=(2,3))
derivative_stdbafu_x_2D[:,0]= -1
derivative_stdbafu_x_2D[0,1]= 1
derivative_stdbafu_x_2D[1,2]= 1

In [ ]:
def derivative_par(i):
    
    node=elem_back[i,:]-1

    local_cart=np.zeros(shape=(2,3))
    local_cart[0,:]=np.radians(xcoord[node])
    local_cart[1,:]=np.radians(ycoord[node])

#   jacobian
    jacobian2D = local_cart[:,1:3].copy()
 
    sub1=local_cart[0,0]
    sub2=local_cart[1,0]

    jacobian2D[0,:]=jacobian2D[0,:]-sub1
    jacobian2D[1,:]=jacobian2D[1,:]-sub2
    
#   check cyclic boundary
    for j in np.arange(2):
        if jacobian2D[0,j]> domain_length/3.0:
            jacobian2D[0,j]=jacobian2D[0,j]-domain_length
        if jacobian2D[0,j]<-domain_length/3.0:
            jacobian2D[0,j]=jacobian2D[0,j]+domain_length

    jacobian2D=jacobian2D*r_earth
    jacobian2D[0,:]=jacobian2D[0,:]*np.mean(np.cos(local_cart[1,:]))

#   inverse
    jacobian2D_inv=np.linalg.inv(jacobian2D)
    derivative_locbafu_x_2D=np.matmul(derivative_stdbafu_x_2D.T,jacobian2D_inv).T
    determinant=np.linalg.det(jacobian2D)

    return(derivative_locbafu_x_2D[0,:],derivative_locbafu_x_2D[1,:],abs(determinant)/2.0)

In [ ]:
bafux_2d,bafuy_2d,voltriangle=zip(*Parallel(n_jobs=-1,batch_size=1000,verbose=10)(delayed(derivative_par)(i)  for i in np.arange(el2d)))

bafux_2d=np.vstack(bafux_2d)
bafuy_2d=np.vstack(bafuy_2d)
voltriangle=np.asarray(voltriangle)

# Geostrophy Calculations
Use the geostrophic equations to calculate zonal and meridional velocity from sea surface height gradients, gravitational acceleration, and the coriolis parameter.

In [ ]:
years=np.arange(1850,2101,1)

In [ ]:
omega=2.0*math.pi/(24.0*60.0*60.0)

def geo_vel_calc(i, run):
    try:
        data=xra.open_dataset('ssp370_'+str(run)+'/zos.fesom.'+str(years[i])+'.nc')
    except:
        data=xra.open_dataset('hist'+str(run)+'/zos.fesom.'+str(years[i])+'.nc')
    ssh_ltm=data.zos.values
    data.close()

    lat_el=np.mean(ycoord[elem[:]-1],axis=1)
    fc=2.0*omega*np.sin(np.radians(lat_el))
    fac=9.8/fc

    ssh_eval=ssh_ltm[:,elem[:]-1]
    u_el=-fac*np.sum(ssh_eval*bafuy_2d,axis=2)
    v_el= fac*np.sum(ssh_eval*bafux_2d,axis=2)
    
    np.save('c6_ssh-uo_daily_elem_r'+str(run)+'_'+str(years[i])+'.npy',u_el)
    np.save('c6_ssh-vo_daily_elem_r'+str(run)+'_'+str(years[i])+'.npy',v_el)

In [ ]:
Parallel(n_jobs=-1,batch_size='auto',verbose=10)(delayed(geo_vel_calc)(i,run)  for i in np.arange(len(years)) for run in [1,2,3,4,5])

# Convert to nodes
convert values from elements (triangle faces) to nodes (triangle vertices) and save the data

In [ ]:
#Determine triangles neighboring each node
def findvals(i):
    vals=np.where((elem.astype(int)-1)==i)
    return vals[0]

node_triags=Parallel(n_jobs=-1,batch_size='auto',verbose=10)(delayed(findvals)(i)for i in np.arange(n2d))

In [ ]:
def node_mean(i,data):
    return(np.sum(data[:,node_triags[i]]*voltriangle[node_triags[i]],axis=1)/np.sum(voltriangle[node_triags[i]]))

In [ ]:
def node_convert(year,run):
    try:
        datedata=xra.open_dataset('hist'+str(run)+'/zos.fesom.'+str(years[year])+'.nc')
    except:
        datedata=xra.open_dataset('ssp370-'+str(run)+'/zos.fesom.'+str(years[year])+'.nc')
        
    dates=datedata.time
    datedata.close()
   
    udata=np.load('c6_ssh-uo_daily_elem_r'+str(run)+'_'+str(years[year])+'.npy')
    vdata=np.load('c6_ssh-vo_daily_elem_r'+str(run)+'_'+str(years[year])+'.npy')
    ures=[]
    vres=[]
    for j in np.arange(n2d):
        ures.append(node_mean(j,udata))
        vres.append(node_mean(j,vdata))
    
    ures=np.vstack(ures)
    vres=np.vstack(vres)
    
    uarr=xra.DataArray(data=ures,dims=['time','ncells'],name='uo',coords={'time':dates},
                    attrs=dict(description='Zonal velocity computed from daily SSH',
                               units='m/s',
                               spatial='nodes on MR mesh. Mean of triangles surrounding each node with area weighting.',
                               simulation='AWI-CM-1-1-MR CMIP6 historic/ssp370 r'+str(run)+'i1p1f1'))
    varr=xra.DataArray(data=vres,dims=['time','ncells'],name='vo',coords={'time':dates},
                    attrs=dict(description='Meridional velocity computed from daily SSH',
                               units='m/s',
                               spatial='nodes on MR mesh. Mean of triangles surrounding each node with area weighting.',
                               simulation='AWI-CM-1-1-MR CMIP6 historic/ssp370 r'+str(run)+'i1p1f1'))
    
    uds=uarr.to_dataset()
    vds=varr.to_dataset()
    
    uds.to_netcdf('c6_ssh-uo_daily_node_r'+str(run)+'_'+str(years[year])+'.nc')
    vds.to_netcdf('c6_ssh-vo_daily_node_r'+str(run)+'_'+str(years[year])+'.nc')

In [ ]:
Parallel(n_jobs=-1,batch_size='auto',verbose=10)(delayed(node_convert)(year,run)  for year in np.arange(len(years)) for run in [1,2,3,4,5])